# Деление на стопы и поиск пробелов

In [ ]:
import os
import sqlite3
from cltk.prosody.latin.hexameter_scanner import HexameterScanner
from cltk.prosody.latin.scansion_formatter import ScansionFormatter
scanner = HexameterScanner()

In [ ]:
def givefiles(path):
    for d, dirs, files in os.walk(path):
        return files

In [ ]:
def readfiles(path, file):
    file = os.path.join(path, file)
    with open(file, 'r', encoding='utf-8') as f:
        text = f.read().split('\n')
        return text

In [ ]:
def scanfeet(text): 
    if not text.startswith('#'):
        listscan = list(scanner.scan(text.strip(' ')))
        listscan[1] = listscan[1].replace(' ','')
        feet = listscan[1].replace(' ','')

        scheme = ScansionFormatter().hexameter(listscan[1])
        if not '|' in scheme:
            if scheme.endswith('-U'):
                scheme = scheme[:-1] + '-'
                scheme = ScansionFormatter().hexameter(scheme)

        #print(scheme)
        feetborders = []
        feet = 0
        for i in scheme:
            if i == '|':
                feetborders.append(feet)
            else:
                feet += 1

        wordborders = []      
        sylstr = ''
        if listscan[5] != '':
            words = listscan[5]
        else:
            words = listscan[5]        
        syllabs = listscan[7].split(', ')

        for number, syl in enumerate(syllabs):
            sylstr = sylstr + syl

            if words.startswith(sylstr) == False:
                #print(sylstr, words)
                wordborders.append(number)
                words = words.split(" ", 1)[-1]
                sylstr = syl

        #print(wordborders)
        #print(feetborders)
        if listscan[3] == True:
            caesurus = []
            for i in wordborders:
                if i == feetborders[0] + 1:
                    caesurus.append('трёхполовинная')
                elif i == feetborders[1] + 1:
                    caesurus.append('пятиполовинная')
                elif i == feetborders[2] + 1:
                    caesurus.append('семиполовинная')
                elif i == feetborders[3]:
                    caesurus.append('буколическая')

                elif len(wordborders) != len(listscan[0].split(' ')) - 1:
                    caesurus = ["Разделы не равны пробелам"]

        else:
            caesurus = ["ошибка"]
            
    else:
        listscan = [text, '', '', '', '', '', '', '']  
        scheme = ''
        feetborders = ''
        wordborders = ''
        caesurus = '#'

    return(listscan, scheme, feetborders, wordborders, caesurus)

In [ ]:
ScansionFormatter().hexameter('-----------U')

In [ ]:
'quīd fremat īmperiūm quīd raucōs tōrqueat aestūs'.startswith('quid')

In [ ]:
print(scanfeet('quid fremat imperium quid raucōs torqueat aestūs '))

In [ ]:
conn = sqlite3.connect('latin_scan4.db')
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS cltk_scan 
(id INTEGER PRIMARY KEY AUTOINCREMENT, file text, original text, scansion text, meter text, 
valid text, syllable_count int, accented text, scansion_notes text, syllables text, 
scheme text, feetborders text, wordborders text, caesurus text)
""")

conn.commit()
conn.close()

In [ ]:
def basefirst(path, filename):
    text = readfiles(path, filename)[3414:3415]
    for i in text:
        all = scanfeet(i)
        tobase = []
        tobase.append(filename)
        tobase.extend(all[0])
        tobase.append(all[1])
        tobase.append(str(all[2]))
        tobase.append(str(all[3]))
        tobase.append(str(all[4]))
        conn = sqlite3.connect('latin_scan4.db')
        cur = conn.cursor()      
        cur.execute('INSERT INTO cltk_scan VALUES (null, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tobase)
        conn.commit()
        conn.close()

In [ ]:
for i in givefiles('C:/Users/M/Desktop/latin/'):
    print(i)
    basefirst('C:/Users/M/Desktop/latin/', i)